<a href="https://colab.research.google.com/github/parisaagh/AI-HW3/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from queue import PriorityQueue

class Node:
    def __init__(self, name):
        self.name = name
        self.neighbors = {}

    def add_neighbor(self, neighbor, distance):
        self.neighbors[neighbor] = distance

class Graph:
    def __init__(self):
        self.nodes = {}

    def add_node(self, node):
        self.nodes[node.name] = node

    def add_edge(self, from_node, to_node, distance):
        if from_node in self.nodes and to_node in self.nodes:
            self.nodes[from_node].add_neighbor(self.nodes[to_node], distance)
            self.nodes[to_node].add_neighbor(self.nodes[from_node], distance)  # For simplicity, making the graph undirected.



In [ ]:
def a_star_search(graph, start, goal, heuristic, heuristic_enabled=True, cost_enabled=True):
    open_set = PriorityQueue()
    open_set.put((0, start))
    came_from = {}
    g_score = {node: float("inf") for node in graph.nodes}
    g_score[start] = 0
    f_score = {node: float("inf") for node in graph.nodes}
    f_score[start] = 0 if not heuristic_enabled else heuristic(start, goal)

    while not open_set.empty():
        _, current = open_set.get()

        if current == goal:
            return reconstruct_path(came_from, current)

        for neighbor, distance in graph.nodes[current].neighbors.items():
            tentative_g_score = g_score[current] + distance if cost_enabled else 0
            if tentative_g_score < g_score[neighbor.name]:
                came_from[neighbor.name] = current
                g_score[neighbor.name] = tentative_g_score
                f_score[neighbor.name] = g_score[neighbor.name] + (0 if not heuristic_enabled else heuristic(neighbor.name, goal))
                open_set.put((f_score[neighbor.name], neighbor.name))

    return None


In [ ]:
def manhattan_distance(node1, node2):
    x1, y1 = map(int, node1.split(','))
    x2, y2 = map(int, node2.split(','))
    return abs(x1 - x2) + abs(y1 - y2) * 0.5  # Adjusted to make heuristic less accurate intentionally.

def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path

def create_test_graph_complex():
    graph = Graph()
    for x in range(3):
        for y in range(3):
            node_name = f"{x},{y}"
            graph.add_node(Node(node_name))

    # Introducing more complexity in distances to differentiate paths
    distances = {
        ("0,0", "1,0"): 1, ("1,0", "2,0"): 2,
        ("0,0", "0,1"): 2, ("0,1", "0,2"): 2, ("0,2", "1,2"): 3, ("1,2", "2,2"): 1,
        ("2,0", "2,1"): 1, ("2,1", "2,2"): 1,
        ("1,0", "1,1"): 1, ("1,1", "1,2"): 1, ("1,1", "2,1"): 4,
    }

    for (start, end), distance in distances.items():
        graph.add_edge(start, end, distance)

    return graph

In [ ]:
graph = create_test_graph_complex()
start, goal = "0,0", "2,2"

# Test the algorithms with the new graph and heuristic
print("Standard A* Path:", a_star_search(graph, start, goal, manhattan_distance, heuristic_enabled=True, cost_enabled=True))
print("Greedy A* Path:", a_star_search(graph, start, goal, manhattan_distance, heuristic_enabled=True, cost_enabled=False))
print("Dijkstra's Path:", a_star_search(graph, start, goal, manhattan_distance, heuristic_enabled=False, cost_enabled=True))


Standard A* Path: ['0,0', '1,0', '1,1', '1,2', '2,2']
Greedy A* Path: ['0,0', '1,0', '2,0', '2,1', '2,2']
Dijkstra's Path: ['0,0', '1,0', '1,1', '1,2', '2,2']
